In [32]:
import pandas as pd

In [33]:
from re import sub
def snake_case(s):
  return '_'.join(
    sub('([A-Z][a-z]+)', r' \1',
    sub('([A-Z]+)', r' \1',
    s.replace('-', ' '))).split()).lower()

In [34]:
file = pd.ExcelFile('results/results.xlsx')

In [35]:
sheet_to_df_map = {}
for sheet_name in file.sheet_names:
    sheet_to_df_map[snake_case(sheet_name)] = file.parse(sheet_name)
    df = sheet_to_df_map[snake_case(sheet_name)]
    df.columns = df.columns.str.strip()
    
sheet_to_df_map['form_responses_2']

,Timestamp,Score,Do you agree with the Brexit result?,Are you in favor of exams starting at a specific time?,Who do you think will win the Premier League this year?,Do you think that women's football is a sport that deserves more attention and support?,What is your opinion on Kanye West?,Do you think 'cancelling' is an appropriate way to deal with controversy?,What is your opinion on influencers?,Kanye West or Taylor Swift?,...,Do you like James Bond films?,Is Beyonce overrated?,Do you think JK Rowling was right to start writing again after her children became adults?,Do you love thunderstorms or hate them?,Are you in favor of Donald Trump becoming the next president of the United States?,"If it had been finished, would you have liked to see Batgirl?",Do you think that Dungeons and Dragons is a good way to spend your time?,Do you think houseplants are beneficial to indoor air quality?,Have you heard of GPT-3 before?,"Please rate your understanding of how language models, such as GPT-3 work?"
0,2022-08-15 16:58:53.005,20,Human,Human,Human,Computer,Computer,Computer,Computer,Computer,...,Computer,Human,Computer,Human,Computer,Human,Human,Computer,No,1
